# Image Generator

In [ ]:
#| default_exp image_generator

In [ ]:
#| eval:false
from Cloud2DImageConverter import spherical_projection as sp
from Cloud2DImageConverter import data
from matplotlib import pyplot as plt
from tqdm import tqdm
from PIL import Image
import numpy as np
import shutil
import time
import os

In [ ]:
#| eval:false
FOLDER_PATH = "../point_clouds/semantic_kitti/"
VELODYNE_PATH = FOLDER_PATH+"velodyne"
LABEL_PATH = FOLDER_PATH+"labels"
RESULTS_FOLDER =  "../results/"

In [ ]:
#| eval:false
def define_range(batch, batch_size, max_len):
    start = batch - batch_size
    end = batch
    if (max_len - batch) < batch_size:
        end = max_len

    return start, end

In [ ]:
#| eval:false
def do_projection(point_cloud):
    projection_dict = {"reflectance": [],"label": [], "depth": []}
    
    for points in point_cloud:
        unlabaled, labaled, depth = sp.spherical_projection(points)
        labaled = sp.colored_matrix_with_label(labaled)
        projection_dict["reflectance"].append(unlabaled)
        projection_dict["label"].append(labaled)
        projection_dict["depth"].append(depth)

    return projection_dict

In [ ]:
#| eval:false
def create_images(projection_dict):    
    for zip_dict in zip(projection_dict["reflectance"], projection_dict["label"], projection_dict["depth"]):
        for i, value in enumerate(projection_dict.keys()): 
            # Aplica o cmap manualmente 
            if value == "reflectance":
                img = zip_dict[i]/np.max(zip_dict[i])
                cmap = plt.get_cmap("jet")
                img = (cmap(img)*255).astype(np.uint8)
                img = Image.fromarray(img)
            elif value == "depth":
                img = zip_dict[i]/np.max(zip_dict[i]) 
                cmap = plt.get_cmap("magma_r")
                img = (cmap(img)*255).astype(np.uint8)
                img = Image.fromarray(img)
            else:
                img = Image.fromarray(zip_dict[i])
                
            folder_type = "/" + value
    
            # Caminho final para salvar
            final_path = os.path.join(RESULTS_FOLDER+folder_type)
    
            # Define o nome do arquivo, tais como 000001, 000002...
            file_number = len(os.listdir(final_path)) + 1
            file_name = f"{final_path}/{file_number:06d}.png"
    
            img.save(file_name)

In [ ]:
#| eval:false
batch_size = 50
max_len = len(os.listdir(VELODYNE_PATH))

if os.path.exists(RESULTS_FOLDER):
    shutil.rmtree(RESULTS_FOLDER)
    
os.makedirs(RESULTS_FOLDER)
os.makedirs(RESULTS_FOLDER+"/reflectance")
os.makedirs(RESULTS_FOLDER+"/label")
os.makedirs(RESULTS_FOLDER+"/depth")

for batch in tqdm(range(batch_size, max_len, batch_size), desc="Batch:"):
    # Define o intervalo de carregamento dos dados
    start, end = define_range(batch, batch_size, max_len)

    # Define as listas contendo os nomes dos arquivos
    velodyne_list = sorted(os.listdir(VELODYNE_PATH))[start:end]
    label_list = sorted(os.listdir(LABEL_PATH))[start:end]

    # Carrega os dados 
    point_cloud = data.load_data(VELODYNE_PATH, LABEL_PATH, velodyne_list, label_list)

    # Realiza a projeção esférica
    projection_dict = do_projection(point_cloud)

    # Cria e salva as imagens
    create_images(projection_dict)

Batch:: 100%|█████████████████████████████████████████████████████████████████| 5/5 [00:26<00:00,  5.24s/it]
